In [1]:
 !pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 69.3 MB/s eta 0:00:00


In [2]:
import pulp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# to display all rows columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [23]:
df = pd.read_csv('ind_sl_t20i_1st_2022_02_24.csv')

In [14]:
df

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1
2,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1
3,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1
4,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1
5,3692,38632,V Iyer,10917,IND,2,AR,53.22,8.5,0,1,50,1
6,3692,38632,Y Chahal,7910,IND,2,BOWL,40.62,8.5,0,1,35,1
7,3692,38632,L Kumara,10928,SL,5,BOWL,34.56,8.5,0,1,35,1
8,3692,38632,R Jadeja,587,IND,2,AR,74.73,9.0,0,1,34,1
9,3692,38632,D Shanaka,8422,SL,5,AR,50.75,9.0,0,1,32,1


In [24]:
def selectStrongTeam(df):
  df = df[df['PlayingXI']==1]
  df = df.join(pd.get_dummies(df[['Team', 'Role']], dtype=int))
  # define decision variable
  pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

  # define objective function
  mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Sel'] for obj in df.index])

  # define constraints
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

  # 11 players
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

  # team constraints
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) >= 4
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_IND']] for obj in df.index) <= 7

  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) >= 4
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Team_SL']] for obj in df.index) <= 7

  # role constraints
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6

  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4
  mo.solve()
  df['pickup_status'] = 0

  for obj in df.index:
    if pickup_status[obj].varValue:
        df['pickup_status'][obj] = 1

  fdf = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)
  return fdf

In [25]:
fdf = selectStrongTeam(df)

In [26]:
fdf

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Team_IND,Team_SL,Role_AR,Role_BAT,Role_BOWL,Role_WK,pickup_status
0,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,1,0,0,1,0,0,1
1,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,0,1,0,1,0,0,1
2,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,1,0,0,1,0,0,1
3,3692,38632,P Nissanka,13682,SL,5,BAT,85.62,9.5,0,1,2,0,0,1,0,1,0,0,1
4,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,1,0,0,0,0,1,1
5,3692,38632,V Iyer,10917,IND,2,AR,53.22,8.5,0,1,50,1,1,0,1,0,0,0,1
6,3692,38632,R Jadeja,587,IND,2,AR,74.73,9.0,0,1,34,1,1,0,1,0,0,0,1
7,3692,38632,D Shanaka,8422,SL,5,AR,50.75,9.0,0,1,32,1,0,1,1,0,0,0,1
8,3692,38632,D Chameera,8393,SL,5,BOWL,78.22,9.0,0,1,30,1,0,1,0,0,1,0,1
9,3692,38632,H Patel,8175,IND,2,BOWL,65.20,8.5,0,1,8,0,1,0,0,0,1,0,1


In [30]:
df = pd.read_csv('ind_sl_t20i_2nd_2022_02_26.csv')


In [31]:
fdf = selectStrongTeam(df)

In [32]:
fdf

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT,Team_IND,Team_SL,Role_AR,Role_BAT,Role_BOWL,Role_WK,pickup_status
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1,1,0,0,1,0,0,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1,0,1,0,1,0,0,1
2,3692,38637,R Sharma,576,IND,2,BAT,87.27,10.5,66,1,1,13,0,1,0,0,1,0,0,1
3,3692,38637,C Asalanka,10934,SL,5,BAT,77.97,8.5,70,1,1,6,0,0,1,0,1,0,0,1
4,3692,38637,I Kishan,10276,IND,2,WK,83.71,8.5,141,1,1,22,0,1,0,0,0,0,1,1
5,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1,1,0,1,0,0,0,1
6,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1,0,1,1,0,0,0,1
7,3692,38637,V Iyer,10917,IND,2,AR,65.32,8.5,50,1,1,12,0,1,0,1,0,0,0,1
8,3692,38637,B Kumar,1726,IND,2,BOWL,63.78,8.5,76,1,1,37,1,1,0,0,0,1,0,1
9,3692,38637,D Chameera,8393,SL,5,BOWL,72.85,9.0,30,1,1,31,1,0,1,0,0,1,0,1


In [33]:
df = pd.read_csv('ind_sl_t20i_3rd_2022_02_27.csv')

In [34]:
fdf = selectStrongTeam(df)

In [35]:
fdf

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,Team_IND,Team_SL,Role_AR,Role_BAT,Role_BOWL,Role_WK,pickup_status
0,3692,38642,S Iyer,9428,IND,2,BAT,88.80,8.5,196.0,2,1,100,1,0,0,1,0,0,1
1,3692,38642,P Nissanka,13682,SL,5,BAT,82.00,9.5,102.0,1,1,-1,0,1,0,1,0,0,1
2,3692,38642,R Sharma,576,IND,2,BAT,81.45,10.5,78.2,1,1,10,1,0,0,1,0,0,1
3,3692,38642,D Gunathilaka,8387,SL,5,BAT,60.51,8.5,56.0,1,1,2,0,1,0,1,0,0,1
4,3692,38642,C Asalanka,10934,SL,5,BAT,57.96,8.5,76.0,1,1,9,0,1,0,1,0,0,1
5,3692,38642,S Samson,8271,IND,2,WK,85.50,8.0,59.0,1,1,35,1,0,0,0,0,1,1
6,3692,38642,R Jadeja,587,IND,2,AR,87.60,9.0,127.0,2,1,31,1,0,1,0,0,0,1
7,3692,38642,D Shanaka,8422,SL,5,AR,82.30,9.0,109.0,2,1,105,0,1,1,0,0,0,1
8,3692,38642,D Chameera,8393,SL,5,BOWL,65.41,9.0,61.0,2,1,31,0,1,0,0,1,0,1
9,3692,38642,L Kumara,10928,SL,5,BOWL,60.44,8.5,87.0,2,1,60,0,1,0,0,1,0,1
